In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from Preprocessing.to_embedding import WordEmbedding
from data_format_phase3 import formatting
from Preprocessing.helper_functions import import_embedding, embedding_matrix_word2vec
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
data = formatting("../phase1_movie_reviews-train.csv")
#data = data[:round(len(data)*.5)]
y = pd.get_dummies(data['polarity'])
X_train, X_dev, y_train, y_dev = train_test_split(data, y, test_size = 0.10, random_state=42)
x

embedding_size = 300 #number of feature weights in embeddings
max_len = 400

:o
1
lel


In [3]:
X_train_nlp, X_dev_nlp = X_train['reviewText'], X_dev['reviewText']
X_train_meta, X_dev_meta = X_train.iloc[:,4:], X_dev.iloc[:,4:]
X_train_meta.head()

,big,worth,new,excellent,little,long,worst,better,funny,old,...,bad,great,young,best,quality,like,set,good,special,affin_score
65743,0,0,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,-0.061224
73053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,-0.106383
36360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0.000000
30998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.313725
71384,0,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.141844


In [4]:

print(X_train_nlp.head())
print(X_train_meta.head())

65743    [god, hate, flick, can't, believe, sister, got, christmas, great, cast, hope, got, nice, paychecks, make, better, movies, scale, 2, hours, would, merely, bad, nearly, three, hours, makes, almost, ...
73053    [somewhere, spirits, three, stooges, decide, death, life, hunt, kill, women, con, serial, killer, dirty, work, sit, get, meantime, ghost, latest, victim, decides, communicate, husband, order, inst...
36360                                                                         [would, like, everyone, except, seeing, theatre, yes, buy, yes, complaint, slight, overuse, product, advertising, placement, miss]
30998    [rugrats, dvd, celebrates, ten, years, fantastic, adventures, nickelodeon, ten, episodes, included, retrospective, fans, cream, crop, episodes, include, angelica, orders, first, cut, ice, bones, v...
71384    [episode, three, star, trek, next, generation's, first, season, first, hand, evidence, new, series, still, long, way, away, developing, complex, involving,

In [5]:
# embedding = WordEmbedding(num_features = embedding_size)

# WordEmbedding.fit(embedding, X_train_nlp)
# WordEmbedding.size(embedding)

In [6]:
#Save word embedding to dataframe
#train_embeddings = WordEmbedding.to_pd(embedding, X_train)

#Save Save embeddings to file
# WordEmbedding.to_file(embedding)

In [7]:
embeddings_index = import_embedding('glove.42B.300d.txt')

## 2. Vectorize text data

In [8]:
#Basic Vectorization of data
#Review data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_nlp)
word_index = tokenizer.word_index

def vectorize(data, tokenizer ,max_len):
    sequences = tokenizer.texts_to_sequences(data)
    padding = pad_sequences(sequences, maxlen = max_len)
    
    return padding

X_train_nlp = vectorize(X_train_nlp, tokenizer , max_len)
X_dev_nlp = vectorize(X_dev_nlp, tokenizer, max_len)

print('Found %s unique tokens.' % len(word_index))
print('Shape of train tensor', X_train_nlp.shape)
print('Shape of dev tensor', X_dev_nlp.shape)

Found 38358 unique tokens.
Shape of train tensor (81000, 400)
Shape of dev tensor (9000, 400)


## 3. Create word vectors with the loaded word2vec model

In [9]:
embedding_matrix, num_words = embedding_matrix_word2vec(word_index, embedding_size, embeddings_index)

### Check train/dev sets

In [10]:
print('Shape of X_train:', X_train_nlp.shape)
print('Shape of y_train:', y_train.shape)
print('Shape of X_test:', X_dev_nlp.shape)
print('Shape of y_test:', y_dev.shape)

Shape of X_train: (81000, 400)
Shape of y_train: (81000, 2)
Shape of X_test: (9000, 400)
Shape of y_test: (9000, 2)


## 5. Define model

In [11]:
import keras

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} ) 
config.gpu_options.allow_growth = True
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [12]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, GRU, Bidirectional, MaxPooling1D, Dropout, Input, concatenate
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras import regularizers

nlp_input = Input(shape=(max_len,), name='nlp_input')
meta_input = Input(shape=(len(X_dev_meta.columns),), name='meta_input')
emb = Embedding(output_dim=embedding_size, input_dim=num_words, input_length=max_len)(nlp_input)
#pool = MaxPooling1D(pool_size=4, strides=1, padding='valid')(emb)
nlp_out = Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=regularizers.l2(0.02)))(emb)
x = concatenate([nlp_out, meta_input])
x = Dense(10, activation='relu')(x)
x = Dense(6, activation='relu')(x)
x = Dense(2, activation='sigmoid')(x)
model = Model(inputs=[nlp_input , meta_input], outputs=[x])
print(model.summary())
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
nlp_input (InputLayer)          (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 400, 300)     11507700    nlp_input[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 256)          439296      embedding_1[0][0]                
__________________________________________________________________________________________________
meta_input (InputLayer)         (None, 23)           0                                            
__________________________________________________________________________________________________
concatenat

In [ ]:
train_sets = [X_train_nlp, X_train_meta]
dev_sets = [X_dev_nlp, X_dev_meta]
history = model.fit(train_sets, y_train, batch_size=256*2, validation_data =(dev_sets, y_dev), epochs=5)

Train on 81000 samples, validate on 9000 samples
Epoch 1/5
81000/81000 [==============================] - 161s 2ms/step - loss: 2.3174 - acc: 0.6429 - val_loss: 0.5070 - val_acc: 0.7908
Epoch 2/5
81000/81000 [==============================] - 158s 2ms/step - loss: 0.4297 - acc: 0.8495 - val_loss: 0.5062 - val_acc: 0.7879
Epoch 3/5
81000/81000 [==============================] - 158s 2ms/step - loss: 0.3490 - acc: 0.8773 - val_loss: 0.3740 - val_acc: 0.8577
Epoch 4/5
81000/81000 [==============================] - 158s 2ms/step - loss: 0.3073 - acc: 0.8875 - val_loss: 0.3965 - val_acc: 0.8329
Epoch 5/5
56320/81000 [===================>..........] - ETA: 46s - loss: 0.2809 - acc: 0.8982

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 3, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 3, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

In [ ]:
loss, accuracy = model.evaluate(train_sets, y_train, verbose=False)
print("Development Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(dev_sets, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)